<a href="https://colab.research.google.com/github/SeohyeonSunny/Topic-Modeling/blob/main/Hierarchical_Topic_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install bertopic

In [ ]:
pip install kiwipiepy

In [ ]:
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
import re
import csv
from sklearn.datasets import fetch_20newsgroups
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic

In [ ]:
kiwi = Kiwi()
stopwords = Stopwords()

In [ ]:
kiwi.add_user_word('인도적', 'NNP', 0)

True

In [ ]:
kiwi.add_user_word('난민법', 'NNG', 0)

True

In [ ]:
file_path = '/content/gdrive/MyDrive/Colab Notebooks/Twitter_refugee_before.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read().splitlines()

In [ ]:
stop_words_2 = '''
이거
그거
그건
그걸
이걸
이건
이걸
그것
거기
카페
누구
무엇
관련
우선
때문'''
stop_words_list2 = stop_words_2.split('\n')

In [ ]:
user_stop_word = stop_words_list2

In [ ]:
extract_pos_list = ["NNG", "NNP", "NNB", "NR", "NP"]

In [ ]:
class CustomTokenizer:
    def __init__(self, kiwi):
        self.kiwi = kiwi

    def __call__(self, text):
        result = list()
        for word in self.kiwi.tokenize(text):
            # 명사이고, 길이가 2이상인 단어이고, 불용어 리스트에 없으면 추가하기
            if word[1] in extract_pos_list and len(word[0]) > 1 and word[0] not in user_stop_word:
                result.append(word[0])
        return result

In [ ]:
custom_tokenizer = CustomTokenizer(kiwi)

In [ ]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=300)

In [ ]:
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 nr_topics= "auto", # 문서를 대표하는 토픽의 갯수
                 top_n_words=100,
                 calculate_probabilities=True)

In [ ]:
topics, probs = model.fit_transform(text)

In [ ]:
model.visualize_topics()

In [ ]:
for i in range(0, 50):
  print(i,'번째 토픽 :', model.get_topic(i))

0 번째 토픽 : [('한국', 0.09664653587881507), ('세계', 0.09552679099525403), ('이민', 0.08463378494728359), ('개도국', 0.06046643308035605), ('각국', 0.0603583818079787), ('정서', 0.0603583818079787), ('구청', 0.0603583818079787), ('완료', 0.06025083758510119), ('확대', 0.06007807590741117), ('설립', 0.06007807590741117), ('위장', 0.06003725334967982), ('난민', 0.05978434041783995), ('효력', 0.05695528240574908), ('징검다리', 0.05675048735750858), ('마디', 0.05653813894168391), ('후진국', 0.05653507115348363), ('시작', 0.055519524194680835), ('영주', 0.05116024018242239), ('사람', 0.050674297727058476), ('시행', 0.04497993456336921), ('""', 0.04432537091693619), ('신청', 0.04234082497104052), ('난민법', 0.03628810334032745), ('비자', 0.004855967523235109), ('연장', 0.004213506466585598), ('의원', 0.0040801867312577345), ('인정', 0.004020041621261269), ('황우여', 0.0032902262790650036), ('탈북자', 0.003151332097878705), ('국제', 0.002990901812903604), ('급증', 0.0027652737767756967), ('유엔', 0.0027146386335793965), ('반기문', 0.0026780593283563026), ('대한민국', 0

In [ ]:
model.visualize_barchart()

In [ ]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,340,-1_난민법_국민_난민_나라,"[난민법, 국민, 난민, 나라, 시행, 반대, 다문화, 최초, 일본, 발의, 제주,...","[나라 없는 국민은 난민일 뿐 !! 국가안보를 위해 제주 해군기지 찬성한다!, 나라..."
1,0,497,0_한국_세계_이민_개도국,"[한국, 세계, 이민, 개도국, 각국, 정서, 구청, 완료, 확대, 설립, 위장, ...","[""개도국까지 확대되는 반이민 정서==>> 세계의 난민들 한국으로 한국으로 !! 그..."
2,1,441,1_미래_외국인_대한민국_일자리,"[미래, 외국인, 대한민국, 일자리, 침식, 천만, 자살, 서민, 문제, 블루오션,...",[내국인에게는 삶의 미래가 없는 대한민국 외국인에게는 삶의 미래가 블루오션인 대한민...
3,2,387,2_난민법_난민_전면_아시아,"[난민법, 난민, 전면, 아시아, 최초, 시행, 제주, 권리, 신청, 뉴스, 지원,...","[""난민법' 7월부터 전면 시행…"""" 아시아 최초"""" "", 난민법' 7월부터 전면 ..."
4,3,102,3_국회_개정안_난민법_개정,"[국회, 개정안, 난민법, 개정, 공청회, 발의, 의원, 통과, 부탁, 내용, 제정...",[[행동하는 다문화 반대] 난민법 개정안 폐기를 위해 법사위 입법조사관과 통화 내용...
5,4,67,4_예멘_시리아_난민_사망,"[예멘, 시리아, 난민, 사망, 아프리카, 제주, 입국, 기구, 여성, 밀입국, 수...","["""""" 나를 땅에 묻지 말아주세요"""" 6살 난 예멘 소년이 온몸에 미사일 파편이..."
6,5,55,5_실체_담론_망국_다문화,"[실체, 담론, 망국, 다문화, 난민법, 문제, 난민, , , , , , , , ,...","[다문화 담론 3) 망국의 법= 난민법의 실체, 다문화 담론 3) 망국의 법= 난민..."
7,6,40,6_국가_찬사_이민자_짐승,"[국가, 찬사, 이민자, 짐승, 스웨덴, 정치인, 중국인, 복지, 자국인, 현실, ...",[한국 짐승 정치인들이 찬사하는 사회복지 난민법이 민 국가스웨덴의 처참한 현실 세계...
8,7,36,7_판결_법원_추방_신청,"[판결, 법원, 추방, 신청, 난민, 취득, 사용, 체류, 조달, 수단, 노동자, ...",[난민법에 의하면 난민 신청만 하면 추방할 수 없음을 법원이 인정 이 젠 난민이 새...
9,8,29,8_난민법_공청회_어필_센터,"[난민법, 공청회, 어필, 센터, 시행령, 시행, 제정, 투표, 생계비, 예산, 법...",[공익법센터 어필(Adv ocates forPublicInterestLaw) : :...


In [ ]:
from scipy.cluster import hierarchy as sch

In [ ]:
linkage_function = lambda x: sch.linkage(x, 'single', optimal_ordering=True)
hierarchical_topics = model.hierarchical_topics(text, linkage_function=linkage_function)

100%|██████████| 13/13 [00:00<00:00, 100.91it/s]


In [ ]:
model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

In [ ]:
topics_to_merge = [[13, 16, 2, 4, 10, 8],
                   [7, 1],
                   [3, 9, 12],
                   [5, 14, 11],
                   [6, 15]]
model.merge_topics(text, topics_to_merge)

IndexError: ignored

In [ ]:
model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

In [ ]:
model.get_topic_info()